In [ ]:

import pandas as pd, numpy as np, os, joblib
from datetime import datetime
from sklearn.metrics import roc_auc_score
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

class StopExecution(Exception):
    def _render_traceback_(self): pass

# 👇 Choose the model to run
MODEL_TO_RUN = "rf_v1"

In [2]:

# STEP 2: Load Data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

C:\Users\Jude Younis\AppData\Local\Temp\ipykernel_28152\495698161.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('train.csv')
C:\Users\Jude Younis\AppData\Local\Temp\ipykernel_28152\495698161.py:3: DtypeWarning: Columns (15,19) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('test.csv')


In [ ]:
# STEP 3: Split features and labels
X = train.drop(columns=['label'])
y = train['label']

X, test = pd.get_dummies(X), pd.get_dummies(test)
X, test = X.align(test, join="left", axis=1, fill_value=0)

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

X = SimpleImputer().fit_transform(X)
test = SimpleImputer().fit_transform(test)
X = StandardScaler().fit_transform(X)
test = StandardScaler().fit_transform(test)



In [ ]:

if MODEL_TO_RUN != "rf_v1":
    raise StopExecution

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
test_preds = np.zeros(test_scaled.shape[0])
auc_scores = []

for train_idx, valid_idx in skf.split(X_scaled, y):
    X_train, X_valid = X_scaled[train_idx], X_scaled[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

    model = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    max_features='sqrt',
    min_samples_leaf=5,
    n_jobs=-1,
    bootstrap=True,
    random_state=42
)

    model.fit(X_train, y_train)

    val_preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, val_preds)
    auc_scores.append(auc)

    test_preds += model.predict_proba(test_scaled)[:, 1] / skf.n_splits

print("Fold AUC scores:", auc_scores)
print("Average Validation AUC:", np.mean(auc_scores))

Fold AUC scores: [0.9331611154593693, 0.9308944194130006, 0.9325550043046467, 0.9331718507250935, 0.9315359895120419]
Average Validation AUC: 0.9322636758828302


In [8]:
# STEP 8: Create Submission File
submission = pd.DataFrame({
    'Id': range(len(test_preds)),
    'Prediction': test_preds
})

submission.to_csv('results_31.csv', index=False)  
